<a href="https://colab.research.google.com/github/fairml-research/Counterfactual_Fairness/blob/main/Counterfactual_Fairness_CRIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import sys, os
os.chdir('/codespace/fairness/Counterfactual_Fairness')

In [77]:
from models.functions import *
from models.adv_vae import ADV_VAE
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

### 1) **IMPORT THE DATA**

In [ ]:
from data_loading.datasets import read_crimes, load_adult, read_law_school

def read_dataset(name, label=None, sensitive_attribute=None, fold=None):
    print("hello there")
    if name == 'crimes':
        y_name = label if label is not None else 'ViolentCrimesPerPop'
        z_name = sensitive_attribute if sensitive_attribute is not None else 'racepctblack'
        fold_id = fold if fold is not None else 1
        return read_crimes(label=y_name, sensitive_attribute=z_name, fold=fold_id)
    elif name=='adult':
        return load_adult()
    elif name == 'law_school':
        y_name = label if label is not None else 'ZFYA'
        z_name = sensitive_attribute if sensitive_attribute is not None else 'race'
        fold_id = fold if fold is not None else 1
        return read_law_school(label=y_name, sensitive_attribute=z_name, fold=fold_id)
    else:
        raise NotImplemented('Dataset {} does not exists'.format(name))
    
X_train, y_train, sensitive, X_test, y_test, sensitivet = read_dataset(name='crimes', fold=1)
X_train, y_train, Z_train, X_test, y_test, Z_test, scaler, scale_df, meanYtrain, stdYtrain= Normalize(X_train, y_train, sensitive, X_test, y_test, sensitivet)

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

### 2) ADVERSARIAL INFERENCE

a) In this part we are going to train a latent space z from X_train, y_train and the sensitive and we will try to recontruct X and Y from the latent space z and the sensitive. In addtion to that there is an adversarial model for mitigating the dependance with the sensitive attribute.

b) In this scenario we have X that is between 0 and 1. Please note that if you want to adapt X for a continuous scenario you need to change this:
 - line 103 in adv_vae.py replace the BCE loss with an MSE one (F.binary_cross_entropy -> F.mse_loss)
 - line 129 in functions.py remove the sigmoid -> (torch.sigmoid(self.fc41(h3)) -> self.fc41(h3)

c) You can see that the betaadv hyerparameter allows to mitigate the dependance with the sensitive (assessed with the HGR at the end of this senction 2). If betaadv =0 the HGR is superior to 0.9 !! and with a lambdaadv it is closer to 0.5

In [ ]:
from torch import nn
sizelat=5
class Adversarial(nn.Module):
    def __init__(self):
        super(Adversarial, self).__init__()
        self.adv_fc1 = nn.Linear(sizelat, 32)
        self.adv_fc2 = nn.Linear(32, 16)
        self.adv_fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = self.adv_fc1(x)
        x = F.tanh(x)
        #x = F.dropout(x, p=0.8)
        x = self.adv_fc2(x)
        x = F.relu(x)
        #x = F.dropout(x, p=0.8)
        x = self.adv_fc3(x)
        #x = torch.sigmoid(x)
        return x
from models.adv_vae import ADV_VAE
#modelVAE=VAE(X_train.shape[1])
from tqdm import trange

batch_size=512
epochs=100
seed=1
log_interval=1
betaX = 1
betaY = 1
#betaKLD = 10
betammd_E =1#1
#betammd_F =0
betaadv=100 #.1
#device="cpu"
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

from tqdm import trange
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.autograd import Variable
from models.functions import VAE
modelVAE=VAE(X_train.shape[1])

ADV_VAE_S0 = ADV_VAE(batch_size, epochs, seed, log_interval, device, nb_features =X_train.shape[1],
                     model = modelVAE, model_adv = Adversarial, sizelat=sizelat,
                     betaX=betaX,betaY=betaY,betammd_E=betammd_E,betaadv=betaadv
                    )
ADV_VAE_S0.fit(X_train, y_train, Z_train)

NameError: name 'X_train' is not defined

In [ ]:
from models.functions import HGR_NN
senstest=torch.tensor(np.expand_dims(Z_test,axis = 1)).float().to(device)
datatest=torch.tensor(X_test.values).float().to(device)
ydatatest= Variable(torch.FloatTensor(np.expand_dims(y_test,axis = 1))).to(device)
recon_X, z, recon_Y, mu, logvar = ADV_VAE_S0.predict(datatest.view(-1, datatest.shape[1]),senstest,ydatatest)
Loss_X = F.binary_cross_entropy(recon_X, datatest.view(-1, datatest.shape[1]), reduction='mean')
Loss_Y = F.mse_loss(recon_Y, ydatatest, reduction='mean')
print("Loss_X TEST", Loss_X.cpu().detach().numpy(), "Loss_Y TEST",Loss_Y.cpu().detach().numpy())
# lambda = 0
HGR_NNP = HGR_NN(Net_HGR(),Net2_HGR(),device, display=False)
print("HGR NN Test",HGR_NNP(senstest , z,200))

In [ ]:
from models.functions import HGR_NN
senstrain=torch.tensor(np.expand_dims(Z_train,axis = 1)).float().to(device)
data=torch.tensor(X_train.values).float().to(device)
ydata= Variable(torch.FloatTensor(np.expand_dims(y_train,axis = 1))).to(device)
recon_X, z, recon_Y, mu, logvar = ADV_VAE_S0.predict(data.view(-1, data.shape[1]),senstrain,ydata)
Loss_X = F.binary_cross_entropy(recon_X, data.view(-1, data.shape[1]), reduction='mean')
Loss_Y = F.mse_loss(recon_Y, ydata, reduction='mean')
print("Loss_X", Loss_X.cpu().detach().numpy(), "Loss_Y",Loss_Y.cpu().detach().numpy() )
# lambda = 0
HGR_NNP = HGR_NN(Net_HGR(),Net2_HGR(),device, display=False)
print("HGR NN Train",HGR_NNP(senstrain , z,200))

### 2) **Unfair Predictor Model** (lambdap=0)

In this second part we are now going to create a predictor model that is counterfactual " UnFair " --> the lambdap of penalization is equal to 0 !

Please note that in the third part we are going to make the predictor model Fairer.

In [ ]:
from models.cf_predictor import CF_PREDICTOR


#device = torch.device("cpu")
#device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1]+1, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)
    def forward(self, x, a):
        x = self.fc1(torch.cat([x,a],1))
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        return x

batch_idx=0
batch_size=256
epochs=1000
seed=1
log_interval=1
lr=0.0001
nb=20
lambdap=0.
CF_PREDICTOR_S0 = CF_PREDICTOR(regressor="mse",batch_size=batch_size, epochs=epochs, seed=seed, log_interval=log_interval, device=device, model = NN, lr=lr, modelVAE= ADV_VAE_S0, nb=nb, lambdap=lambdap)
CF_PREDICTOR_S0.fit(X_train, y_train, Z_train)

In [ ]:
senstest=torch.tensor(np.expand_dims(Z_test,axis = 1)).float().to(device)
datatest=torch.tensor(X_test.values).float().to(device)
ydatatest= Variable(torch.FloatTensor(np.expand_dims(y_test,axis = 1))).to(device)

Yhat_test=CF_PREDICTOR_S0.predict(datatest,senstest)
MSE_Test = F.mse_loss(Yhat_test,ydatatest)
print("MSE TEST:",MSE_Test.cpu().detach().numpy())

In [ ]:
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

nb = 1000
Uni = np.random.uniform(Z_train.min(),Z_train.max(),nb)
#Unif_X = torch.tensor(np.random.uniform(Z_train.min(),Z_train.max(),Z_test.shape[0]*nb), dtype=torch.float32).to(device)
Unif_X = torch.tensor(torch.tensor(np.expand_dims(Uni,axis=1)).repeat( X_test.shape[0],1), dtype=torch.float32).to(device)
data_X = torch.tensor(np.repeat(X_test.values, nb,0), dtype=torch.float32).to(device)
Y_X    = torch.tensor(np.repeat(y_test.values, nb,0), dtype=torch.float32).to(device)
recon_X_aprime, z_aprime, recon_Y_aprime, mu_aprime, logvar_aprime = ADV_VAE_S0(data_X.view(-1, data_X.shape[1]),Unif_X,Y_X)
Z_train_X = torch.tensor(np.repeat(Z_test.values, nb,0), dtype=torch.float32).to(device)
recon_X_a, z_a, recon_Y_a, mu_a, logvar_a = ADV_VAE_S0.predict(data_X, Z_train_X , Y_X)

predY_a_prime = CF_PREDICTOR_S0.predict(recon_X_aprime,Unif_X).cpu().detach().numpy()
predY_a = CF_PREDICTOR_S0.predict(recon_X_a,Z_train_X.unsqueeze(1)).cpu().detach().numpy()
print('CF Value:  {:9f}'.format(np.mean((predY_a_prime-predY_a)**2),8))

For calculating the CF we generate 1000 possible counterfactual observations for each instance of X_test. For this, we repeat the X instances for each counterfactual and aggrate with a mean average loss.
We observe in this scenario that the CF is pretty unfair .. we will try to mitigate this in the third part

### 3) **Fair Predictor Model** (lambdap=1)

In this third part we are now going to create a predictor model that is counterfactual Fair.  At each batch and for each instance, we will generate 20 counterfactual observations (nb=20) from the inference model (from the step 1)

In [ ]:
from models.cf_predictor import CF_PREDICTOR

#device = torch.device("cpu")
#device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1]+1, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)
    def forward(self, x, a):
        x = self.fc1(torch.cat([x,a],1))
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        return x

batch_idx=0
batch_size=256
epochs=1000
seed=1
log_interval=1
lr=0.0001
nb=20
lambdap=1.
CF_PREDICTOR_S0 = CF_PREDICTOR(regressor="mse",batch_size=batch_size, epochs=epochs, seed=seed, log_interval=log_interval, device=device, model = NN, lr=lr, modelVAE= ADV_VAE_S0, nb=nb, lambdap=lambdap)
CF_PREDICTOR_S0.fit(X_train, y_train, Z_train)

In [ ]:
senstest=torch.tensor(np.expand_dims(Z_test,axis = 1)).float().to(device)
datatest=torch.tensor(X_test.values).float().to(device)
ydatatest= Variable(torch.FloatTensor(np.expand_dims(y_test,axis = 1))).to(device)

Yhat_test=CF_PREDICTOR_S0.predict(datatest,senstest)
MSE_Test = F.mse_loss(Yhat_test,ydatatest)
print("MSE TEST:",MSE_Test.cpu().detach().numpy())

In [ ]:
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

nb = 1000
Uni = np.random.uniform(Z_train.min(),Z_train.max(),nb)
#Unif_X = torch.tensor(np.random.uniform(Z_train.min(),Z_train.max(),Z_test.shape[0]*nb), dtype=torch.float32).to(device)
Unif_X = torch.tensor(torch.tensor(np.expand_dims(Uni,axis=1)).repeat( X_test.shape[0],1), dtype=torch.float32).to(device)
data_X = torch.tensor(np.repeat(X_test.values, nb,0), dtype=torch.float32).to(device)
Y_X    = torch.tensor(np.repeat(y_test.values, nb,0), dtype=torch.float32).to(device)
recon_X_aprime, z_aprime, recon_Y_aprime, mu_aprime, logvar_aprime = ADV_VAE_S0(data_X.view(-1, data_X.shape[1]),Unif_X,Y_X)
Z_train_X = torch.tensor(np.repeat(Z_test.values, nb,0), dtype=torch.float32).to(device)
recon_X_a, z_a, recon_Y_a, mu_a, logvar_a = ADV_VAE_S0.predict(data_X, Z_train_X , Y_X)

predY_a_prime = CF_PREDICTOR_S0.predict(recon_X_aprime,Unif_X).cpu().detach().numpy()
predY_a = CF_PREDICTOR_S0.predict(recon_X_a,Z_train_X.unsqueeze(1)).cpu().detach().numpy()
print('CF Value:  {:9f}'.format(np.mean((predY_a_prime-predY_a)**2),8))